**TITLE:** MULTI-AGENT INTERVIEWING SYSTEM

**DEVELOPERS:**

------

# Setup Instructions for Jupyter Notebook

This notebook installs essential packages for working with LangChain, OpenAI, and other data handling tools. 

### Important Notes:
- **Google Colab Users**: If you are using Google Colab, ensure to install `google-colab` specific packages. 
- **GPU Configuration**: If using Google Colab, you can enable GPU for faster performance by going to:
  - **Runtime** > **Change runtime type** > **Hardware accelerator** and selecting **GPU**.
  
---

## Step 1: Install General Utilities and Google Colab Packages


In [ ]:
# Install general utilities and widgets
%pip install pandas opendatasets nest_asyncio ipywebrtc ipywidgets IPython 

In [ ]:
# Only run this cell if using google-colab, else skip it
%pip install google-colab

---

## Step 2: Install OpenAI, LangChain, and Related Tools
These packages are necessary for using OpenAI’s language models and LangChain's toolkit for search, document processing, and data handling.

---


In [ ]:
# OpenAI and related LangChain tools
%pip install openai langchain_openai

# LangChain Community Tools for search and document handling
%pip install langchain_community

# Typing extensions and Pydantic
%pip install typing_extensions pydantic

# LangGraph and experimental LangChain tools
%pip install langgraph langchain_experimental

In [ ]:
# For agent tools
%pip install pypdf wikipedia duckduckgo-search playwright

!playwright install

---

## Step 3: Database Utilities, SQLAlchemy, and FAISS for Vector Storage

- **Database Utilities**: Install SQLAlchemy for database interactions.
- **FAISS**: Choose `faiss-cpu` for CPU environments or `faiss-gpu` if you've enabled GPU support on Colab.

---


In [ ]:
# Database utilities and SQLAlchemy
%pip install SQLAlchemy

# FAISS for vector storage and retrieval
%pip install faiss-cpu

In [ ]:
!where python
!pip show playwright


## General Imports
This cell includes the essential imports needed to use LangChain, OpenAI, and other data handling tools in any Jupyter Notebook or Python environment.


In [ ]:
# General imports for data handling, display, and LangChain functionality
import os
import opendatasets as od
import nest_asyncio

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display, clear_output
import ipywidgets as widgets

import openai
from openai import OpenAI
from langchain_openai import ChatOpenAI

# LangChain and related tools
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool

# LangChain Agents and supporting libraries
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, trim_messages
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from pydantic import BaseModel
from typing import Annotated, Literal, Sequence, List
import functools
import operator
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import create_react_agent
from langchain_experimental.tools import PythonREPLTool
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.memory import MemorySaver


## Google Colab Specific Imports
This cell should be run only if you're using Google Colab.


In [ ]:
# Google Colab specific imports
from google.colab import output
from google.colab import userdata
from google.colab import files


In [ ]:
nest_asyncio.apply()

# Get API Keys

In [ ]:
# Check if running on Google Colab
try:
    # Retrieve API key from Google Colab userdata (if stored there)
    open_ai_api_key = userdata.get('OPENAI_API_KEY')
except:
    # Not running on Google Colab; prompt for API key input or retrieve from environment variables
    open_ai_api_key = os.getenv('OPENAI_API_KEY') or input("Enter your OpenAI API key: ")

# Set the API key as an environment variable for universal access within the notebook
os.environ['OPENAI_API_KEY'] = open_ai_api_key

# Confirm setup
if open_ai_api_key:
    print(f"API key successfully set: {open_ai_api_key}")
else:
    print("API key not set. Please check your setup.")


In [ ]:
# Todo: 
# Need to have an alternative that grabs a HuggingFace API key and interfaces with free models there (Llama-3-8B)

# Create Tools

In [ ]:
from langchain_core.tools import tool


## 1. Speech-to-text

This tool allows the user to record speech and converts it to a text using OpenAI Whisper model.



In [ ]:
client = OpenAI()

In [ ]:
# for colab
output.enable_custom_widget_manager()

In [ ]:
def setup_audio_recorder():
    camera = CameraStream(constraints={'audio': True, 'video': False})
    recorder = AudioRecorder(stream=camera)
    display(recorder)
    return recorder

In [ ]:
def save_recording(recorder):
    audio_data = recorder.audio.value
    if audio_data:
        with open("recording.webm", "wb") as f:
            f.write(audio_data)
        return "recording.webm"
    else:
        print("No audio data was captured. Please try again.")
        return None

In [ ]:
def convert_to_wav(input_filename, output_filename="my_recording.wav"):
    if input_filename and os.path.exists(input_filename):
        os.system(f"ffmpeg -i {input_filename} -ac 1 -f wav {output_filename} -y -hide_banner -loglevel panic")
        if os.path.exists(output_filename):
            return output_filename
        else:
            print("Conversion failed.")
            return None
    else:
        print("Input file does not exist.")
        return None

In [ ]:
def transcribe_audio(filename):
    with open(filename, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
    print("")
    print("Transcription:", transcription.text)
    return transcription.text

In [ ]:
def record_and_transcribe_candidate_answer():
    """Record and transcribe a candidate's answer on interviewers' questions."""
    # Set up the recorder
    recorder = setup_audio_recorder()

    # Create a save button
    print("")
    save_button = widgets.Button(description="Save Recording")

    # This dictionary will store the transcribed text
    transcription_result = {}

    # Define the callback function for the save button
    def on_save_clicked(button):
        # Save the recording
        webm_file = save_recording(recorder)
        if webm_file:
            # Convert to wav format
            wav_file = convert_to_wav(webm_file)
            if wav_file:
                # Transcribe the audio and store the result
                transcription_result['text'] = transcribe_audio(wav_file)

    save_button.on_click(on_save_clicked)
    display(save_button)

    # Return the transcription result dictionary
    return transcription_result

In [ ]:
# Todo:
# Try to do live transcription, rather than recording a file. 
# Take a look at https://gist.github.com/Vaibhavs10/a48d141534cc8d877937d421bb828d8e
# and https://github.com/VRSEN/langchain-agents-tutorial/blob/main/main.py

# FOSS alternative pipeline, that doesn't rely on OpenAI models
# Using HF free API instead 
# Something like https://github.com/nyrahealth/CrisperWhisper?tab=readme-ov-file#31-usage-with--transformers

## 2. Text Input

In [ ]:
def setup_text_input():
    text_input = widgets.Textarea(
        placeholder="Type your answer here...",
        description="Answer:",
        layout=widgets.Layout(width='500px', height='100px')
    )
    display(text_input)
    return text_input

In [ ]:
def submit_text_input(text_widget):
    user_text = text_widget.value
    if user_text.strip():
        print("\nInput:\n", user_text)
        return user_text
    else:
        print("No input was provided. Please type your answer and try again.")
        return None

In [ ]:
def record_and_submit_text():
    """Record a candidate's text answer on interviewers' questions which require written output like code."""
    # Set up the text input widget
    text_widget = setup_text_input()

    # Create a submit button
    print("")
    submit_button = widgets.Button(description="Save Answer")

    # This variable will store the submitted text
    submission_result = {}

    # Define the callback function for the submit button
    def on_submit_clicked(button):
        # Capture the user's text input and store it in the dictionary
        submission_result['text'] = submit_text_input(text_widget)

    submit_button.on_click(on_submit_clicked)
    display(submit_button)

    # Wait for user input to be submitted
    return submission_result

## 3. CV Reader

CV Reader for PDF and DOCX files.

Instead of CV you can upload your LinkedIn profile extract, which can be exported in a PDF format.

This tools can be easily changed to any file reading service, e.g., Azure DI, LlamaParse, custom parsing with PyPdf, etc.

In [ ]:
# colab version

def upload_and_filter_file():
    # Upload a single file
    uploaded = files.upload()

    # Check if only one file was uploaded
    if len(uploaded) != 1:
        print("Please upload exactly one file.")
        return None

    # Get the uploaded file name and data
    file_name, file_data = next(iter(uploaded.items()))

    # Check if the file is .pdf or .docx
    if not file_name.endswith(('.pdf', '.docx')):
        print("Invalid file type. Please upload only .pdf or .docx files.")
        return None

    # Save the file directly to the /content/ directory
    file_path = f'/content/{file_name}'

    return file_path

cv_file_path = upload_and_filter_file()

In [ ]:
# local jupyter notebook

cv_file_path = r'C:\Users\DMA\Downloads\CV - 2024-1.pdf'

In [ ]:
cv_file_path

In [ ]:
def create_cv_retriever(file_path, k):
    pages = []

    if file_path.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith('.docx'):
        loader = Docx2txtLoader(file_path)
    else:
        raise ValueError("Unsupported file type.")

    for page in loader.load():
        pages.append(page)

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(pages)

    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(texts, embeddings)

    retriever = db.as_retriever(search_kwargs={"k": k})

    return retriever

In [ ]:
cv_retriever = create_cv_retriever(cv_file_path, 5)

In [ ]:
cv_tool = create_retriever_tool(
    cv_retriever,
    "search_candidate_info",
    "Searches and returns candidate's profile with experience, education, and skills.",
)

In [ ]:
# todo: 
# Free alternative for embeddings that doesn't use OpenAI

## 4. Hiring Company Info Scraper

In [ ]:
def get_wikipedia_content(query):
    """Fetches content from Wikipedia based on a query."""
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    wikipedia_content = wikipedia.run(query)
    return wikipedia_content

In [ ]:
def get_websites_links(query):
    """Fetches a list of website links based on a search query using DuckDuckGo."""
    search = DuckDuckGoSearchResults(output_format="list")
    search_results = search.invoke(query)
    return [result["link"] for result in search_results]

In [ ]:
def load_websites_content(websites):
    """Loads the HTML content of a list of websites."""
    content_list = []
    for website in websites:
        loader = AsyncChromiumLoader([website])
        html_content = loader.load()
        content_list.append(html_content)
    return content_list

In [ ]:
def transform_html_content(html_content_list, tags = ["span", "p", "b", "h3", "h4"]):
    """Transforms HTML content to extract specific tags using BeautifulSoup."""
    transformed_content = []
    bs_transformer = BeautifulSoupTransformer()
    for html in html_content_list:
        docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=tags)
        for doc in docs_transformed:
            transformed_content.append(doc.page_content)
    return transformed_content

In [ ]:
def get_web_content(query):
    """Main function to gather content from Wikipedia and websites based on a query."""
    content = []

    wikipedia_content = get_wikipedia_content(query)
    content.append(wikipedia_content)

    website_links = get_websites_links(f"What is {query}?")

    html_content_list = load_websites_content(website_links)

    transformed_content = transform_html_content(html_content_list)

    content.extend(transformed_content)
    return content

In [ ]:
query = "Deloitte Company"
websites_content = get_web_content(query)
websites_content

In [ ]:
def create_company_info_retriever(websites_content, k):
    docs = []

    for website_content in websites_content:
        doc = Document(page_content=website_content)
        docs.append(doc)

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings()  # need a FOSS alternative
    db = FAISS.from_documents(texts, embeddings)

    retriever = db.as_retriever(search_kwargs={"k": k})

    return retriever

In [ ]:
company_info_retriever = create_company_info_retriever(websites_content, 5)

In [ ]:
# todo: update this tool so it gets correct data, this is copied from the cv

company_info_tool = create_retriever_tool(
    company_info_retriever,
    "search_company_info",
    "Searches and returns company's profile with company's details to be considered by HR Specialist.",
)

## 5. Querying a Dataset

This is an optional tool for enhancing the process of hard skills review.

The dataset can be changed depending on the needs of users.

In [ ]:
ds = "https://www.kaggle.com/datasets/syedmharis/software-engineering-interview-questions-dataset"

In [ ]:
def get_kaggle_ds(dataset_url):
    od.download(dataset_url)

In [ ]:
# Load CSV

# Set the file path to the downloaded data and the encoding of the file
file_path = r"C:\Users\DMA\Downloads\Software Questions.csv"
encoding = "ISO-8859-1"  # default English encoding

loader = CSVLoader(file_path=file_path, encoding=encoding)
docs = loader.load()


In [ ]:
# Define text splitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)


### 5.1a Using OpenAI Embeddings 

In [ ]:
def create_questions_dataset_retriever(texts, k):
    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(texts, embeddings)

    retriever = db.as_retriever(search_kwargs={"k": k})

    return retriever

In [ ]:
create_questions_dataset_retriever(texts=texts, k=5)

### 5.1b Using HuggingFace Embeddings 

To represent each chunk as a high-dimensional vector, we’ll use Hugging Face's pre-trained model sentence-transformers/all-MiniLM-L6-v2. This model is efficient and well-suited for generating text embeddings.


We’ll define a simple helper class to handle embedding generation using the Hugging Face model.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

class HuggingFaceEmbeddings:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        # Load the model and tokenizer from Hugging Face
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def embed_texts(self, texts):
        # Generate embeddings for each text
        embeddings = []
        for text in texts:
            inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
            with torch.no_grad():
                outputs = self.model(**inputs)
                embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
        return np.array(embeddings)

Now, let’s generate embeddings for each of the text chunks.

In [ ]:
# Initialize the embedding model
embeddings_model = HuggingFaceEmbeddings()

# Generate embeddings for each chunk of text
embeddings = embeddings_model.embed_texts([text.page_content for text in texts])

After this step, `embeddings` will contain a vector representation of each document chunk.

To make our embeddings searchable, we’ll use FAISS to create an index. This allows us to find the most similar embeddings to any query.

In [ ]:
import faiss

# Initialize the FAISS index
embedding_dim = embeddings.shape[1]  # Dimension of embeddings
faiss_index = faiss.IndexFlatL2(embedding_dim)

# Add the embeddings to the FAISS index
faiss_index.add(embeddings)

Finally, we’ll define a `retriever` function that, given a query, will embed it and retrieve the most similar document chunks from the FAISS index.

In [ ]:
def retriever(query, texts, embeddings_model, faiss_index, k=5):
    # Generate embedding for the query
    query_embedding = embeddings_model.embed_texts([query])[0]
    
    # Search FAISS index for the top-k similar chunks
    distances, indices = faiss_index.search(np.array([query_embedding]), k)
    
    # Retrieve the corresponding text chunks
    results = [texts[i].page_content for i in indices[0]]
    return results


For testing it:

In [ ]:
# Define your query
query = "What is the topic of interest?"

# Call the retriever with the required arguments
results = retriever(query, texts, embeddings_model, faiss_index, k=5)

# Print the top results
print("Top similar chunks:")
for i, result in enumerate(results, 1):
    print(f"{i}. {result}")


### 5.2 Define the tool for agents

In [ ]:
# todo: update this tool so its usable by agents

@tool
questions_database_tool = create_retriever_tool(
    create_questions_dataset_retriever,
    "search_subject_matter_questions",
    "Searches and returns subject matter questions for checking hard skills.",
)

-----

# Initialize Agents

In [ ]:
# Todo:
# Need to test this with OAI key
# Test each of the tools are working

# Create LangGraph agents, give them roles, assign interactions and tools to each

# Implement user-agent interaction
# LangGraph - https://github.com/langchain-ai/langgraph/blob/main/docs/docs/how-tos/human_in_the_loop/wait-user-input.ipynb

# Add a FOSS alternative for models

In [ ]:
llm = ChatOpenAI(model_name="gpt-4o")  # need a FOSS alternative

In [ ]:
def display_input_form_with_return():
    # Capture inputs
    print("Invoice input")
    print("")
    voice_input = record_and_transcribe_candidate_answer()
    print("")
    print("")
    print("Text input")
    print("")
    written_input = record_and_submit_text()

    # Define what happens on submit
    def on_submit(button):
        clear_output()
        print("Submitted successfully. Moving to the next step...")

    # Create the submit button and link to the on_submit action
    print("")
    print("================================================")
    print("Please, click submit button to send your answers")
    print("")
    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(on_submit)

    display(submit_button)

    if submit_button:
      return voice_input, written_input

In [ ]:
voice, text_input = display_input_form_with_return()

In [ ]:
answer = f"Answer: {voice.get('text', '') if voice else ''}\n\n{text_input.get('text', '') if text_input else ''}"
answer

In [ ]:
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

In [ ]:
memory = MemorySaver()

In [ ]:
builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")
graph = builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import display, Image
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
config = {"configurable": {"thread_id": "1"}}

In [ ]:
input_message = {"type": "user", "content": answer}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [ ]:
input_message = {"type": "user", "content": answer}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

# Agents (DMA)

In [ ]:
from langgraph.prebuilt import ToolNode

from pydantic import BaseModel

In [ ]:
llm = ChatOpenAI(model_name="gpt-4o")  # need a FOSS alternative

In [ ]:
tools = []
tool_node = ToolNode(tools)

In [ ]:
# We are going "bind" all tools to the model
# We have the ACTUAL tools from above, but we also need a mock tool to ask a human
# Since `bind_tools` takes in tools but also just tool definitions,
# We can define a tool definition for `ask_human`
class AskHuman(BaseModel):
    """Ask the human a question"""

    question: str
  
llm = llm.bind_tools(tools + [AskHuman])

In [ ]:
# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # If tool call is asking Human, we return that node
    # You could also add logic here to let some system know that there's something that requires Human input
    # For example, send a slack message, etc
    elif last_message.tool_calls[0]["name"] == "AskHuman":
        return "ask_human"
    # Otherwise if there is, we continue
    else:
        return "continue"


In [ ]:
# Define the function that calls the llm
def call_model(state):
    messages = state["messages"]
    response = llm.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [ ]:
# We define a fake node to ask the human
def ask_human(state):
    pass

Build the graph

In [ ]:
from langgraph.graph import END, StateGraph

In [ ]:
# Define a new graph
workflow = StateGraph(MessagesState)

# Define the three nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)
workflow.add_node("ask_human", ask_human)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

In [ ]:
# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # We may ask the human
        "ask_human": "ask_human",
        # Otherwise we finish.
        "end": END,
    },
)

In [ ]:
# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# After we get back the human response, we go back to the agent
workflow.add_edge("ask_human", "agent")

In [ ]:
# Set up memory
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
# We add a breakpoint BEFORE the `ask_human` node so it never executes
app = workflow.compile(checkpointer=memory, interrupt_before=["ask_human"])

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(
    content="Ask the user where they are"
)
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

In [ ]:
tool_call_id = app.get_state(config).values["messages"][-1].tool_calls[0]["id"]

from langchain_core.messages import ToolMessage
tool_message = [ToolMessage(tool_call_id=tool_call_id, content="san francisco")]

# We now update the state
# Notice that we are also specifying `as_node="ask_human"`
# This will apply this update as this node,
# which will make it so that afterwards it continues as normal
app.update_state(config, {"messages": tool_message}, as_node="ask_human")

# We can check the state
# We can see that the state currently has the `agent` node next
# This is based on how we define our graph,
# where after the `ask_human` node goes (which we just triggered)
# there is an edge to the `agent` node
app.get_state(config).next

In [ ]:
for event in app.stream(None, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

In [ ]:
# Retrieve the current state of the application
state = app.get_state(config).values

# Access the list of messages from the state
messages = state["messages"]

# Iterate through each message and print its content
for message in messages:
    print(f"{message.type.capitalize()} Message: {message.content}")


-----
# Stretch goal: TTS

Example:

Define model and TTS pipelines

In [ ]:
from transformers import pipeline

# Load the TTS model
tts_pipeline = pipeline("text-to-speech", model="espnet/kan-bayashi_ljspeech_vits")


Generate and Play Text with TTS in Real-Time

Create a loop where the language model generates text in small chunks. Each chunk will be converted to speech and played immediately.

In [ ]:
import IPython.display as ipd

def generate_and_play_text(prompt, max_chunks=5, chunk_size=50):
    generated_text = ""
    
    # Generate text in chunks
    for _ in range(max_chunks):
        # Generate a chunk of text
        output = text_generator(prompt + generated_text, max_new_tokens=chunk_size, do_sample=True)
        new_text = output[0]["generated_text"][len(prompt + generated_text):]
        
        # Append the new text to the generated text
        generated_text += new_text
        print(new_text)  # Print the generated text chunk

        # Generate TTS for the current chunk
        audio = tts_pipeline(new_text)

        # Autoplay the audio chunk in the notebook
        ipd.display(ipd.Audio(audio["wav"], autoplay=True))
        
        # Add a short delay to simulate real-time generation if needed
        # time.sleep(1)  # Uncomment if you want to control the timing

# Example usage
generate_and_play_text("Once upon a time,")
